In [20]:
import sys
sys.path.insert(0, '..')

import polars as pl
from pathlib import Path

# Load df_sample_split from interim folder
interim_dir = Path('../data/interim')
df = pl.read_csv(interim_dir / 'df_sample_split.csv')

print(f"df_sample_split loaded!")
print(f"Shape: {df.shape}")
print(f"\nColumn names: {df.columns}")
print(f"\nData types:")
print(df.schema)
print(f"\nFirst few rows:")
print(df.head())

df_sample_split loaded!
Shape: (412, 15)

Column names: ['id', 'session', 'electoralTerm', 'firstName', 'lastName', 'politicianId', 'speechContent', 'factionId', 'documentUrl', 'positionShort', 'positionLong', 'date', 'speech_length', 'paragraph_number', 'paragraph_length']

Data types:
Schema([('id', Int64), ('session', Int64), ('electoralTerm', Int64), ('firstName', String), ('lastName', String), ('politicianId', Int64), ('speechContent', String), ('factionId', Int64), ('documentUrl', String), ('positionShort', String), ('positionLong', String), ('date', String), ('speech_length', Int64), ('paragraph_number', Int64), ('paragraph_length', Int64)])

First few rows:
shape: (5, 15)
┌────────┬─────────┬────────────┬───────────┬───┬────────────┬────────────┬────────────┬───────────┐
│ id     ┆ session ┆ electoralT ┆ firstName ┆ … ┆ date       ┆ speech_len ┆ paragraph_ ┆ paragraph │
│ ---    ┆ ---     ┆ erm        ┆ ---       ┆   ┆ ---        ┆ gth        ┆ number     ┆ _length   │
│ i64   

In [2]:
# trying the sentiment API

from germansentiment import SentimentModel

model = SentimentModel()

texts = [
    "Mit keinem guten Ergebniss","Das ist gar nicht mal so gut",
    "Total awesome!","nicht so schlecht wie erwartet",
    "Der Test verlief positiv.","Sie fährt ein grünes Auto."]
       
result = model.predict_sentiment(texts)
print(result)

classes, probabilities = model.predict_sentiment(["das ist super"], output_probabilities = True) 
print(classes, probabilities)


c:\Users\olive\miniconda3\envs\parl_speech\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\olive\miniconda3\envs\parl_speech\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\olive\.cache\huggingface\hub\models--oliverguhr--german-sentiment-bert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. 

c:\Users\olive\miniconda3\envs\parl_speech\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\olive\miniconda3\envs\parl_speech\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\olive\.cache\huggingface\hub\models--oliverguhr--german-sentiment-bert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. 

KeyboardInterrupt: 

In [22]:
# Cut paragraphs longer than 300 words
import polars as pl

def trim_to_max_words(text, max_words=300):
    """
    Trim text to maximum number of words.
    If text exceeds max_words, cut it off at that point.
    """
    if text is None or text == "":
        return text
    
    words = text.split()
    if len(words) > max_words:
        return ' '.join(words[:max_words])
    return text

print("Trimming paragraphs longer than 300 words...")

# Apply trimming to speechContent
df_trimmed = df.with_columns(
    pl.col('speechContent').map_elements(
        lambda x: trim_to_max_words(x, max_words=300),
        return_dtype=pl.Utf8
    ).alias('speechContent')
)

# Calculate word counts for statistics
df_trimmed = df_trimmed.with_columns(
    pl.col('speechContent').str.split(by=' ').list.len().alias('word_count')
)

print(f"\nTrimming complete!")
print(f"Total rows: {df_trimmed.shape[0]}")
print(f"\nWord count statistics after trimming:")
print(f"  Min words: {df_trimmed['word_count'].min()}")
print(f"  Max words: {df_trimmed['word_count'].max()}")
print(f"  Mean words: {df_trimmed['word_count'].mean():.1f}")
print(f"  Median words: {df_trimmed['word_count'].median():.1f}")

print(f"\nFirst few rows after trimming:")
print(df_trimmed.select(['id', 'word_count', 'speechContent']).head(5))

Trimming paragraphs longer than 300 words...

Trimming complete!
Total rows: 412

Word count statistics after trimming:
  Min words: 1
  Max words: 300
  Mean words: 51.8
  Median words: 26.0

First few rows after trimming:
shape: (5, 3)
┌────────┬────────────┬─────────────────────────────────┐
│ id     ┆ word_count ┆ speechContent                   │
│ ---    ┆ ---        ┆ ---                             │
│ i64    ┆ u32        ┆ str                             │
╞════════╪════════════╪═════════════════════════════════╡
│ 738998 ┆ 7          ┆ Frau Präsidentin! Meine Damen … │
│ 738998 ┆ 47         ┆ Kollege Heveling, ich bin auf … │
│ 738998 ┆ 77         ┆ Der Anlass für unsere heutige … │
│ 738998 ┆ 39         ┆ Zu den Zahlen. Sie wissen, 12 … │
│ 738998 ┆ 18         ┆ Allein die Ermittlungsstelle K… │
└────────┴────────────┴─────────────────────────────────┘


In [26]:
# Calculate sentiment for each paragraph
from germansentiment import SentimentModel
import polars as pl

print("Loading sentiment model...")
model = SentimentModel()

print(f"Processing {len(df_trimmed)} paragraphs for sentiment analysis...")

# Extract texts
texts = df_trimmed['speechContent'].to_list()

# Predict sentiments with probabilities
sentiments = []
probabilities_list = []

for i, text in enumerate(texts):
    if i % 50 == 0:
        print(f"  Processed {i}/{len(texts)} paragraphs...")
    
    classes, probabilities = model.predict_sentiment([text], output_probabilities=True)
    sentiments.append(classes[0])
    probabilities_list.append(probabilities[0])

print(f"  Processed {len(texts)}/{len(texts)} paragraphs - Complete!")

# Add sentiment columns to dataframe
df_sentiment = df_trimmed.with_columns([
    pl.Series('sentiment_class', sentiments),
    pl.Series('sentiment_probabilities', probabilities_list, dtype=pl.Object)
])

print(f"\nSentiment analysis complete!")
print(f"Total paragraphs: {len(df_sentiment)}")

# Show sentiment distribution
print(f"\nSentiment class distribution:")
print(df_sentiment['sentiment_class'].value_counts().sort('count', descending=True))

# Extract individual probabilities for better visualization
print(f"\nFirst few results:")
print(df_sentiment.select(['id', 'word_count', 'sentiment_class', 'sentiment_probabilities']).head(5))

Loading sentiment model...
Processing 412 paragraphs for sentiment analysis...
  Processed 0/412 paragraphs...
Processing 412 paragraphs for sentiment analysis...
  Processed 0/412 paragraphs...
  Processed 50/412 paragraphs...
  Processed 50/412 paragraphs...
  Processed 100/412 paragraphs...
  Processed 100/412 paragraphs...
  Processed 150/412 paragraphs...
  Processed 150/412 paragraphs...
  Processed 200/412 paragraphs...
  Processed 200/412 paragraphs...
  Processed 250/412 paragraphs...
  Processed 250/412 paragraphs...
  Processed 300/412 paragraphs...
  Processed 300/412 paragraphs...
  Processed 350/412 paragraphs...
  Processed 350/412 paragraphs...
  Processed 400/412 paragraphs...
  Processed 400/412 paragraphs...
  Processed 412/412 paragraphs - Complete!

Sentiment analysis complete!
Total paragraphs: 412

Sentiment class distribution:
shape: (3, 2)
┌─────────────────┬───────┐
│ sentiment_class ┆ count │
│ ---             ┆ ---   │
│ str             ┆ u32   │
╞══════════

In [28]:
# Save sentiment analysis results
import polars as pl
from pathlib import Path
import json

# Define output path
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Convert probabilities to JSON strings for storage
df_to_save = df_sentiment.with_columns(
    pl.col('sentiment_probabilities').map_elements(
        lambda x: json.dumps(x) if isinstance(x, list) else str(x),
        return_dtype=pl.Utf8
    ).alias('sentiment_probabilities_json')
).drop('sentiment_probabilities').rename({'sentiment_probabilities_json': 'sentiment_probabilities'})

# Save as Parquet
output_file = output_dir / 'df_sample_sentiment.parquet'
df_to_save.write_parquet(output_file)

print(f"✓ Sentiment analysis results saved!")
print(f"  File: {output_file}")
print(f"  Rows: {len(df_to_save)}")
print(f"  Columns: {len(df_to_save.columns)}")

# Also save as CSV for easier viewing
csv_output = output_dir / 'df_sample_sentiment.csv'
df_to_save.write_csv(csv_output)
print(f"\n✓ CSV version saved for easy viewing")
print(f"  File: {csv_output}")

# Display summary statistics
print(f"\n📊 Sentiment Analysis Summary:")
print(f"  Total paragraphs analyzed: {len(df_sentiment)}")
print(f"\n  Sentiment Distribution:")
sentiment_counts = df_sentiment['sentiment_class'].value_counts().sort('count', descending=True)
for row in sentiment_counts.iter_rows(named=True):
    percentage = (row['count'] / len(df_sentiment)) * 100
    print(f"    {row['sentiment_class']:10s}: {row['count']:3d} ({percentage:5.1f}%)")

print(f"\n✓ Data ready for further analysis!")

✓ Sentiment analysis results saved!
  File: ..\data\processed\df_sample_sentiment.parquet
  Rows: 412
  Columns: 18

✓ CSV version saved for easy viewing
  File: ..\data\processed\df_sample_sentiment.csv

📊 Sentiment Analysis Summary:
  Total paragraphs analyzed: 412

  Sentiment Distribution:
    neutral   : 261 ( 63.3%)
    positive  : 107 ( 26.0%)
    negative  :  44 ( 10.7%)

✓ Data ready for further analysis!
